# 10. 3D PointCloud reconstruction

* Point Cloud Library - https://pointclouds.org/downloads/ - C++ (has bad python ports)
* Open3D - Python - http://www.open3d.org/ - written for python
*  PDAL libraryfor pipelining

* data from  - http://graphics.stanford.edu/data/3Dscanrep/
  - download armadillo scans
* https://github.com/intel-isl/Open3D/tree/master/examples/python/pipelines

In [14]:
import open3d as o3d
import numpy as np
import os
import sys
import copy

source = o3d.io.read_point_cloud("/home/miro/Documents/Armadillo_scans/ArmadilloOnface_90.ply")
target = o3d.io.read_point_cloud("/home/miro/Documents/Armadillo_scans/ArmadilloOnface_120.ply")

In [15]:
o3d.visualization.draw_geometries([source, target])

In [16]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

### Fast Point Feature Histograms (FPFH) descriptors

* https://pcl.readthedocs.io/projects/tutorials/en/latest/fpfh_estimation.html
* orientacia normal okolia, normala je kolmica na povrch ktoru vieme urcite pre kazdy bod resp skupinu bodov,
* orientacia je definovana tromi uhlami voci refrencnemu bodu

In [17]:
def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [18]:
voxel_size = 0.001
source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
o3d.visualization.draw_geometries([source_down])

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5

    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,distance_threshold)
    return result

result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)

RegistrationResult with fitness=6.869601e-02, inlier_rmse=8.995172e-04, and correspondence_set size of 167
Access transformation to get result.


In [19]:
draw_registration_result(source, target, result_ransac.transformation)

In [20]:
threshold= 0.002
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source_down, target_down, threshold, result_ransac.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)

Apply point-to-point ICP
RegistrationResult with fitness=7.241341e-01, inlier_rmse=6.498156e-04, and correspondence_set size of 6460
Access transformation to get result.
Transformation is:
[[ 8.66008205e-01 -5.79618526e-03 -4.99996193e-01  8.69398571e-04]
 [ 6.70822876e-03  9.99977499e-01  2.66528035e-05 -6.79814130e-05]
 [ 4.99984789e-01 -3.37717039e-03  8.66027601e-01  2.62462350e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [21]:
draw_registration_result(source, target, reg_p2p.transformation)

In [22]:
def merge_pt_clouds(source, target, transform):
    pcd_combined = o3d.geometry.PointCloud()

    source_T = source.transform(transform)

    pcd_combined += target
    pcd_combined += source_T
    pcd_combined =pcd_combined.voxel_down_sample(0.001)

    return pcd_combined

pt_cloud  = merge_pt_clouds(source,target,reg_p2p.transformation)
o3d.visualization.draw_geometries([pt_cloud])

In [23]:
print(pt_cloud)

PointCloud with 15095 points.


In [24]:
from glob import glob

paths = glob("/home/miro/Documents/Armadillo_scans/ArmadilloOnface*.ply")
out_cloud = copy.deepcopy(pt_cloud)
voxel_size = 0.001


In [25]:
print(out_cloud)

for fp in paths:
    source = o3d.io.read_point_cloud(fp)
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(out_cloud, voxel_size)

    result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
    print(result_ransac)
    if np.asarray(result_ransac.correspondence_set).shape[0]>25:
        out_cloud = merge_pt_clouds(source,out_cloud,result_ransac.transformation)
        print(out_cloud)

PointCloud with 15095 points.
RegistrationResult with fitness=1.975634e-03, inlier_rmse=1.061673e-03, and correspondence_set size of 6
Access transformation to get result.
RegistrationResult with fitness=3.663004e-03, inlier_rmse=9.282469e-04, and correspondence_set size of 10
Access transformation to get result.
RegistrationResult with fitness=1.612298e-01, inlier_rmse=8.121315e-04, and correspondence_set size of 430
Access transformation to get result.
PointCloud with 16470 points.
RegistrationResult with fitness=1.973165e-03, inlier_rmse=1.220768e-03, and correspondence_set size of 5
Access transformation to get result.
RegistrationResult with fitness=2.637528e-03, inlier_rmse=1.120509e-03, and correspondence_set size of 7
Access transformation to get result.
RegistrationResult with fitness=3.024803e-03, inlier_rmse=9.513905e-04, and correspondence_set size of 5
Access transformation to get result.
RegistrationResult with fitness=2.650762e-03, inlier_rmse=8.355918e-04, and correspon

In [26]:
o3d.visualization.draw_geometries([out_cloud])

 1. Zrekonstruovat bunny point cloud, lubovolnou metodou.
    * http://graphics.stanford.edu/pub/3Dscanrep/bunny.tar.gz